# 2. CLASSIFICATORE LINEARE SVM CON INPUT DI N-GRAMMI

Classificatore basato su  SVM che prende in input una matrice di features basata su n-grammi di 3 tipi:
1. Caratteri
2. Token
3. Part of Speech

***

### Preprocessing dei dati

Dobbiamo ottenere un vettore di features, basato sulle occorrenze di n-grammi all'interno delle frasi, per ciascun post. 

In [2]:
import pandas as pd
import os

In [3]:
#creiamo un dataframe che abbia 4 colonne, id (identificativo frase), word, lemma
def create_set(type):
    annotated_posts = []
    for doc in os.listdir("../data/UD_annotation"):
        if type in doc: 
            doc_tokens = []
            doc_path = "../data/UD_annotation/" + doc
            for line in open(doc_path, "r", encoding="utf-8"):
                splitted = line.strip().split("\t")
                if splitted[0].isdigit() and "-" not in splitted[0]:
                    word=splitted[1]
                    lemma=splitted[2]
                    pos=splitted[3]
                    new_token = {'word': word, 'lemma': lemma, 'pos': pos}
                    doc_tokens.append(new_token)
            annotated_posts.append(doc_tokens)
    return annotated_posts

annotation_tr = create_set("training")
annotation_ts = create_set("test")


In [4]:
print(annotation_tr[0])

[{'word': 'Le', 'lemma': 'il', 'pos': 'DET'}, {'word': '5', 'lemma': '5', 'pos': 'NUM'}, {'word': 'sgradevoli', 'lemma': 'sgradevole', 'pos': 'ADJ'}, {'word': 'realtà', 'lemma': 'realtà', 'pos': 'NOUN'}, {'word': 'di', 'lemma': 'di', 'pos': 'ADP'}, {'word': 'cui', 'lemma': 'cui', 'pos': 'PRON'}, {'word': 'Berlusconi', 'lemma': 'Berlusconi', 'pos': 'PROPN'}, {'word': 'dovrebbe', 'lemma': 'dovere', 'pos': 'AUX'}, {'word': 'render', 'lemma': 'rendere', 'pos': 'VERB'}, {'word': 'si', 'lemma': 'si', 'pos': 'PRON'}, {'word': 'personalmente', 'lemma': 'personalmente', 'pos': 'ADV'}, {'word': 'conto', 'lemma': 'conto', 'pos': 'NOUN'}, {'word': '<URL>', 'lemma': '<URL>', 'pos': 'PROPN'}, {'word': 'Mario', 'lemma': 'Mario', 'pos': 'PROPN'}, {'word': 'Monti', 'lemma': 'Monti', 'pos': 'PROPN'}, {'word': 'non', 'lemma': 'non', 'pos': 'ADV'}, {'word': 'usa', 'lemma': 'usare', 'pos': 'VERB'}, {'word': 'mezzi', 'lemma': 'mezzo', 'pos': 'ADJ'}, {'word': 'termini', 'lemma': 'termine', 'pos': 'NOUN'}]


In [5]:
#contiamo il numero di post(sample) per training e test
print(f'Training: {len(annotation_tr)}')
print(f'Test: {len(annotation_ts)}')

Training: 3977
Test: 872


In [16]:
def count_ngrams(post, ft_type, n, char=False):
    all_ngrams = {}
    allwords = []
    if not char:
        for word in post:
            allwords.append(word[ft_type])
        post_length = len(allwords)
    else:
        for word in post:
            allwords.append(word['word'])
        allwords = " ".join(allwords)
        post_length = len(allwords)-1
    for i in range(len(allwords)-(n-1)):
        new_ngram = allwords[i:i+n]
        ngram_id = f'{ft_type}_{n}_'+'_'.join(new_ngram)
        if ngram_id not in all_ngrams:
            all_ngrams[ngram_id] = 1
        else:
            all_ngrams[ngram_id]+=1
    #normalizzare in base alla post_length
    print(post_length)
    return all_ngrams
        

print(count_ngrams(annotation_tr[0], "word", 2, True))



118
{'word_2_L_e': 1, 'word_2_e_ ': 3, 'word_2_ _5': 1, 'word_2_5_ ': 1, 'word_2_ _s': 2, 'word_2_s_g': 1, 'word_2_g_r': 1, 'word_2_r_a': 1, 'word_2_a_d': 1, 'word_2_d_e': 2, 'word_2_e_v': 1, 'word_2_v_o': 1, 'word_2_o_l': 1, 'word_2_l_i': 1, 'word_2_i_ ': 7, 'word_2_ _r': 2, 'word_2_r_e': 3, 'word_2_e_a': 1, 'word_2_a_l': 2, 'word_2_l_t': 1, 'word_2_t_à': 1, 'word_2_à_ ': 1, 'word_2_ _d': 2, 'word_2_d_i': 1, 'word_2_ _c': 2, 'word_2_c_u': 1, 'word_2_u_i': 1, 'word_2_ _B': 1, 'word_2_B_e': 1, 'word_2_e_r': 4, 'word_2_r_l': 1, 'word_2_l_u': 1, 'word_2_u_s': 2, 'word_2_s_c': 1, 'word_2_c_o': 2, 'word_2_o_n': 5, 'word_2_n_i': 2, 'word_2_d_o': 1, 'word_2_o_v': 1, 'word_2_v_r': 1, 'word_2_e_b': 1, 'word_2_b_b': 1, 'word_2_b_e': 1, 'word_2_e_n': 2, 'word_2_n_d': 1, 'word_2_r_ ': 1, 'word_2_s_i': 1, 'word_2_ _p': 1, 'word_2_p_e': 1, 'word_2_r_s': 1, 'word_2_s_o': 1, 'word_2_n_a': 1, 'word_2_l_m': 1, 'word_2_m_e': 2, 'word_2_n_t': 3, 'word_2_t_e': 2, 'word_2_t_o': 1, 'word_2_o_ ': 2, 'word_2_ 